## Crear index

In [1]:
import pyterrier as pt 
import os
import matplotlib.pyplot as plt

# Cargar dataset Vaswani (alias “npl”)
dataset = pt.datasets.get_dataset("vaswani")                  # 11 429 docs, 93 queries, 2 083 qrels

# Indexar con IterDictIndexer
index_path = "./vaswani_index"
if os.path.exists(index_path + "/data.properties"):
    tfidf = pt.BatchRetrieve(index_path + "/data.properties")
else:
    indexer = pt.index.IterDictIndexer(index_path)
    indexref = indexer.index(dataset.get_corpus_iter())
    tfidf = pt.BatchRetrieve(indexref, wmodel="TF_IDF")

/tmp/ipykernel_21231/4093397133.py:11: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  tfidf = pt.BatchRetrieve(index_path + "/data.properties")
Java started (triggered by Retriever.__init__) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


## Experimentos

In [2]:
topics = dataset.get_topics()[:3]  # queries
qrels = dataset.get_qrels()[:3] # juicios de relevancia
topics_rank = tfidf.transform(topics)   # ejecuta las consultas

topics[:3]

,qid,query
0,1,measurement of dielectric constant of liquids ...
1,2,mathematical analysis and design details of wa...
2,3,use of digital computers in the design of band...


### Global

In [ ]:
# Experimento ámbito global: P@10, AP, nDCG@10 + gráfica R-P en 11 puntos
global_exp = pt.Experiment(
    [tfidf],    # tf-idf
    dataset.get_topics(), 
    dataset.get_qrels(),
    names=["TF-IDF"], 
    eval_metrics=["P_10", "map", "ndcg_cut_10"],
    perquery=False  # si es True, retorna los resultados de cada query; si es False, retorna el promedio global
)
print(global_exp)

     name      P_10       map  ndcg_cut_10
0  TF-IDF  0.358065  0.283614     0.434968


: 

In [ ]:
recall = pt.Utils.evaluate(topics_rank, qrels, metrics=["iprec_at_recall_0.00", "iprec_at_recall_0.10", "iprec_at_recall_0.20", "iprec_at_recall_0.30", "iprec_at_recall_0.40",
                                       "iprec_at_recall_0.50", "iprec_at_recall_0.60", "iprec_at_recall_0.70", "iprec_at_recall_0.80", "iprec_at_recall_0.90",
                                       "iprec_at_recall_1.00"])

precision_values = [
    recall["iprec_at_recall_0.00"],
    recall["iprec_at_recall_0.10"],
    recall["iprec_at_recall_0.20"],
    recall["iprec_at_recall_0.30"],
    recall["iprec_at_recall_0.40"],
    recall["iprec_at_recall_0.50"],
    recall["iprec_at_recall_0.60"],
    recall["iprec_at_recall_0.70"],
    recall["iprec_at_recall_0.80"],
    recall["iprec_at_recall_0.90"],
    recall["iprec_at_recall_1.00"]
]

recall_points = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

plt.figure(figsize=(8, 5))
plt.plot(recall_points, precision_values, marker="o", linestyle="--", color="b")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Curva R-Precision (11 puntos estándar)")
plt.grid(True)
plt.xticks(recall_points)
plt.show()


In [ ]:
# recalls = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
# iprec_at_recall = [ IPrec(recall=r) for r in recalls ] # no andaba la eval_metrics "iprec_at_recall" de pyTerrier, asi que la hice a mano

pr_exp = pt.Experiment(
    [tfidf],
    topics,
    qrels,
    names=["TF-IDF"],
    eval_metrics=["iprec_at_recall_0.00", "iprec_at_recall_0.10", "iprec_at_recall_0.20", "iprec_at_recall_0.30", "iprec_at_recall_0.40", "iprec_at_recall_0.50", "iprec_at_recall_0.60", "iprec_at_recall_0.70", "iprec_at_recall_0.80", "iprec_at_recall_0.90", "iprec_at_recall_1.00"],
    perquery=False
)



### Individual (por query)

In [ ]:
indv_exp = pt.Experiment(
    [tfidf],
    topics,
    qrels,
    names=["TF-IDF"],
    eval_metrics=["P_10","map","ndcg_cut_10"],
    perquery=True
)
df_indv = indv_exp.T
# df_indv index incluirá cada qid, con columnas P@10, map, ndcg_cut_10
print(df_indv.head())

In [ ]:
import seaborn as sns
sns.boxplot(data=df_indv[['P@10','map','ndcg_cut_10']])
plt.title('Distribución por consulta de P@10, AP, nDCG@10')
plt.show()